# Tutorial for OpenAI API
目次
- OpenAI APIのインストール
- 環境変数の設定
- シンプルな文章生成
- オープンデータセットを用いた文章分類
- 課題1. 文章分類タスクの評価
- Extra課題

注意点
- この演習はGoogle Colaboratyで実行することを想定しています

## OpenAI APIのインストール
- [公式ドキュメント](https://platform.openai.com/docs/api-reference?lang=python)
- [GitHub](https://github.com/openai/openai-python)

In [1]:
# ライブラリのインストール
!pip install openai==1.3.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


## 環境変数の設定
左のタブの`シークレット`から`OPENAI_API_KEY`を設定


<img width=500 src="https://github.com/HarmoLab/aitr/blob/main/2023/exercise_04/colab_secret.png?raw=true">

In [2]:
from google.colab import userdata
from openai import OpenAI

client = OpenAI(
    api_key= userdata.get('OPENAI_API_KEY')
)

In [3]:
# promptから料金を出力する関数
def calculate_credit(model:str, input_tokens:int, output_tokens:int) -> float:
    yen_rate = 150 # $1 = 150円で計算
    if model == "gpt-3.5-turbo-1106":
        input_credit = (0.001 * yen_rate) / 1000
        output_credit = (0.002 * yen_rate) / 1000
    elif model == "gpt-4-1106-preview":
        input_credit = (0.01 * yen_rate) / 1000
        output_credit = (0.03 * yen_rate) / 1000
    else:
        assert False, f"モデル名: {model}の料金計算はできません。modelは'gpt-3.5-turbo-1106',または'gpt-4-1106-preview'を使用してください。"
    total_credit = round(input_credit * input_tokens + output_credit * output_tokens, 2)
    print(f"使用したモデル: {model}, 料金: {total_credit}円")
    return total_credit


## シンプルな文章生成
- 公式ドキュメント
    - 文章生成(text-generation): https://platform.openai.com/docs/guides/text-generation


In [12]:
user_prompt = "ChatGPTについて簡潔に3文で教えてください。" # ここを修正
model = "gpt-3.5-turbo-1106"

response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": user_prompt
        },
    ],
    model=model,
    max_tokens=500,
    n=1,
    temperature=1,

)

In [ ]:
print("--- レスポンス全体 ---")
print(response)

calculate_credit(model, response.usage.prompt_tokens, response.usage.completion_tokens)

print()
print("--- 質問 ---")
print(user_prompt)
print("--- 回答 ---")
print(response.choices[0].message.content)

### chat.completions.createに使用する引数について
公式ドキュメント: https://platform.openai.com/docs/api-reference/chat/create

### 主要な引数
- model (必須)
    - 使用する学習モデル (本演習では"gpt-3.5-turbo-1106"を使用)
- messages (必須)
    - role
        - system: アシスタントの動作を設定
        - assistant: アシスタントの望ましい動作を設定 (ユーザも作成可)
        - user: ユーザの指示
    - content
        - roleに対して入力する文章
- max_tokens: int or null
    - 生成するトークンの最大数。出力の長さを制限することが可能
- n: int or null
    - 生成するレスポンスの数 (デフォルト: 1)
- temperature: number or null
    - 0～2の間で指定
    - 0に近づくにつれて決定論的になり、2に近づくにつれて生成文が多様でランダムになる（デフォルト: 1）
        - 詳しい説明: https://techblog.a-tm.co.jp/entry/2023/04/24/181232

## オープンデータセットを用いた文章分類
- 今回使用するデータセット
    - Amazon Reviews Multi:
https://huggingface.co/datasets/mteb/amazon_reviews_multi/viewer/ja

In [4]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00


In [ ]:
import datasets
# Amazonレビューのデータセット(日本語)をダウンロード
# 今回はダウンロード時間を減らすために検証データ (validation)のみ取得
dataset = datasets.load_dataset("mteb/amazon_reviews_multi", "ja", split="validation")
print(dataset)

In [ ]:
# 実際のデータセットを確認
dataset[0]["text"]

In [7]:
system_prompt = "以下のレビュー文を評価してください"
user_prompt = dataset[0]["text"] # レビュー文のみ

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": user_prompt
        },
    ],
    model="gpt-3.5-turbo",
)

In [ ]:
print("--- 全体の出力 ---")
print(response)

calculate_credit(model, response.usage.prompt_tokens, response.usage.completion_tokens)

print()
print("--- 質問 ---")
print(user_prompt)
print("--- 回答 ---")
print(response.choices[0].message.content)

In [8]:
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)

# forでループするために関数化 (RateLimit対策のためtenacityのデコーダを採用)
# 参考: https://cookbook.openai.com/examples/how_to_handle_rate_limits
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def simple_completion(system_prompt:str, user_prompt:str):
    model_name = "gpt-3.5-turbo-1106"
    response = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": user_prompt
            },
        ],
        model=model_name,
        timeout=15
    )
    # 料金の計算
    credit = calculate_credit(model_name, response.usage.prompt_tokens, response.usage.completion_tokens)
    return response, credit

## 課題1. 文章分類タスクの評価
system_promptを変更してレビュー文章を用いた評価値推定を行いましょう。

In [ ]:
k = 10 # 10件のレビュー文に対する評価
shuffled_dataset = dataset.shuffle(seed=70) # データセットをシャッフル
review_list = shuffled_dataset[:k]["text"]
label_list = shuffled_dataset[:k]["label"]
print(review_list)
print(label_list)

# 課題1: 正答率と評価値を上げるためのsystem_promptを考えましょう。
system_prompt = """
以下のレビュー文を評価してください
"""

predict_label_list = [] # -1(出力形式のエラー), 0, 1, 2, 3, 4
total_credit = 0
for i, (review, label) in enumerate(zip(review_list, label_list)):
    print(f"\n^^^^^^^^^^ {i+1}件目のレビュー ^^^^^^^^^^")
    print("--- レビュー文 ---")
    print(review)
    print("--- 実際の評価 ---")
    print(label)
    response, credit = simple_completion(system_prompt, review)
    total_credit += credit
    print("--- ChatGPTの回答 ---")
    print(response)
    print(response.choices[0].message.content)

    try:
        response_int = int(response.choices[0].message.content)
        predict_label_list.append(response_int)
    except ValueError:
        print("--- Error ---")
        print("回答がint型に変換できませんでした。")
        predict_label_list.append(-1)
        continue
print(f"合計金額: {total_credit}円")

In [ ]:
print("--- 実際の評価 ---")
print(label_list)
print("--- 予測した評価--- ")
print(predict_label_list)

# 評価
## 実際の評価と予測した評価の差の合計を評価値 (loss)
## 予測した評価が-1（出力形式のエラー）の場合は+6

validation_score = 0 # 評価値
correct_answer = 0 # 正解数
for actual, predict in zip(label_list, predict_label_list):
    if not actual == predict:
        if predict == -1:
            validation_score += 6
            continue
        validation_score += abs(actual - predict)
    else:
        correct_answer += 1
print("\n--- 正答率 ---")
print(f"{correct_answer/len(label_list)*100} %")
print("--- 評価値 (低いほうが良い結果) ---")
print(validation_score)

## Extra課題


In [ ]:
user_prompt = "2泊3日の札幌観光旅行のプランを考えてください。"

# Extra課題
system_prompt = """
ユーザの質問に回答してください。
"""

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": user_prompt
        },
    ],
    model="gpt-3.5-turbo",
)

In [ ]:
print("--- 全体の出力 ---")
print(response)

calculate_credit(model, response.usage.prompt_tokens, response.usage.completion_tokens)

print()
print("--- 質問 ---")
print(user_prompt)
print("--- 回答 ---")
print(response.choices[0].message.content)